Import pandas, numpy, matplotlib, IPython, figure display settings.

In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from IPython import display
display.set_matplotlib_formats('retina')
%matplotlib inline

Load the .csv file of published geochemical data for the Columbia River Basalt Group using pandas.

In [7]:
df = pd.read_csv('GEOROC_Compilation_ColumbiaRiverPlateau_11_13_20.csv')
df.head()

/srv/conda/envs/notebook/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (0,3,13,18,22,26,28,30,31,33,93,94) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Year,CITATION,SAMPLE NAME,UNIQUE_ID,LOCATION,ELEVATION (MIN.),ELEVATION (MAX.),SAMPLING TECHNIQUE,DRILLING DEPTH (MIN.),DRILLING DEPTH (MAX.),...,ENSTATITE(MOL%),FAYALITE(MOL%),FORSTERITE(MOL%),FERROSILITE(MOL%),ORTHOCLASE(MOL%),TEPHROITE(MOL%),ULVOSPINEL(MOL%),WOLLASTONITE(MOL%),AGE(MA),AGE_PB206_U238(MA)
0,1966,[1594] CROCKET J. H. (1966),samp. B-5-D,NaN,YELLOWSTONE-SNAKE RIVER PLAIN VOLCANIC PROVINC...,NaN,NaN,OUTCROP,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1973,[1726] WALKER G. W. (1973),samp. GWW-1-7,NaN,YELLOWSTONE-SNAKE RIVER PLAIN VOLCANIC PROVINC...,NaN,NaN,OUTCROP,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1973,[1726] WALKER G. W. (1973),samp. GWW-3-71,NaN,YELLOWSTONE-SNAKE RIVER PLAIN VOLCANIC PROVINC...,NaN,NaN,OUTCROP,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1973,[1726] WALKER G. W. (1973),samp. EL-2-71,NaN,YELLOWSTONE-SNAKE RIVER PLAIN VOLCANIC PROVINC...,NaN,NaN,OUTCROP,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1970,[1728] OSAWA M. (1970),samp. PG-1,11271-PG-1,YELLOWSTONE-SNAKE RIVER PLAIN VOLCANIC PROVINC...,NaN,NaN,OUTCROP,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Clean up the data file for any missing values.

Address any inconsistent reporting of data (i.e. reporting FeO vs Fe2O3, all data will be converted to FeO*), then normalize major element data to 100%.

Chondrite normalize trace element data.

Use the groupby function to sort the data based on stratographic units.

Create a map illustrating the geographical location of data points, colorcode by stratographic unit.

To analyze the evolution of geochemistry over the emplacement of the Columbia River Basalt Group, plot Total Alkali-Silica (TAS) diagram and REE spider diagram grouped by stratographic unit, and plot REE spider diagram group.

Calculate and plot Europium (Eu*) anomaly grouped by stratographic unit.

Calculate and plot Dysprosium (Dy) anomaly grouped by stratographic unit.

To analyze the inferred volatile content over the emplacement of the Columbia River Basalt Group, create a multi-panel plot using trace elements as proxies: Ce for H2O, Nb for CO2, Dy for S.